In [1]:
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split

dataset_path = "/Users/shun/Desktop/IERG4080/aclImdb"

#example 1
def read_reviews(directory):
    reviews = []
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            review = file.read()
            reviews.append(review)
    return reviews

train_positive_reviews = read_reviews(os.path.join(dataset_path,"train", "pos"))
train_negative_reviews = read_reviews(os.path.join(dataset_path,"train", "neg"))

test_positive_reviews = read_reviews(os.path.join(dataset_path, "test", "pos"))
test_negative_reviews = read_reviews(os.path.join(dataset_path, "test", "neg"))

# train_positive_reviews = pd.DataFrame(train_positive_reviews)
# train_positive_reviews.insert(1,"1","pos")

# train_negative_reviews = pd.DataFrame(train_negative_reviews)
# train_negative_reviews.insert(1,"1","neg")

# test_positive_reviews = pd.DataFrame(test_positive_reviews)
# test_positive_reviews.insert(1,"1","pos")

# test_negative_reviews = pd.DataFrame(test_negative_reviews)
# test_negative_reviews.insert(1,"1","neg")

train_data = train_positive_reviews + train_negative_reviews
test_data = test_positive_reviews + test_negative_reviews
full_data = train_data + test_data

#exmaple 2
# Initialize empty lists for reviews and labels
reviews = []
labels = []

# Read the raw training data
train_pos_dir = os.path.join(dataset_path, "train", "pos")
train_neg_dir = os.path.join(dataset_path, "train", "neg")

for filename in os.listdir(train_pos_dir):
    with open(os.path.join(train_pos_dir, filename), "r", encoding="utf-8") as file:
        review = file.read()
        reviews.append(review)
        labels.append("pos")
        
for filename in os. listdir(train_neg_dir):
    with open(os.path.join(train_neg_dir, filename), "r", encoding="utf-8") as file:
        review = file.read()
        reviews.append (review)
        labels.append("neg")

# Read the raw test data
test_pos_dir = os.path.join(dataset_path, "test", "pos")
test_neg_dir = os.path.join(dataset_path, "test", "neg")

for filename in os.listdir(test_pos_dir):
    with open(os.path.join(test_pos_dir, filename), "r" , encoding="utf-8") as file:
        review = file.read()
        reviews.append(review)
        labels.append("pos")
        
for filename in os.listdir(test_neg_dir):
    with open(os.path.join(test_neg_dir, filename), "r", encoding="utf-8") as file:
        review = file.read()
        reviews.append(review)
        labels.append("neg")

# Create a DataFrame, whatever you like to store the combined data
data = pd.DataFrame({"review": reviews, "sentiment": labels})
# Optionally, you can save the combined dataset to a CSV file
data.to_csv("fulldata.csv", index=False) # Replace with the desired output file path

# train_data = train_pos_dir + train_neg_dir
# test_data = test_pos_dir + test_neg_dir
# full_data = train_data + test_data

random.shuffle(full_data)

num_reviews = len(full_data)
num_train_reviews = int(num_reviews * 0.7)
num_test_reviews = num_reviews - num_train_reviews

train_reviews = full_data[:num_train_reviews]
test_reviews = full_data[num_train_reviews:]

train_data, test_data = train_test_split(full_data, test_size =0.3, random_state=42)


positive_data = train_positive_reviews + test_positive_reviews

num_positive_train = sum(1 for review in train_reviews if review in positive_data)
num_negative_train = num_train_reviews - num_positive_train
positive_to_negative_ratio_train = num_positive_train / num_negative_train

num_positive_test = sum(1 for review in test_reviews if review in positive_data)
num_negative_test = num_test_reviews - num_positive_test
positive_to_negative_ratio_test = num_positive_test / num_negative_test

print(f"Training Set: Positive-to-Negative Ratio = {positive_to_negative_ratio_train:.2f}")
print(f"Testing Set: Positive-to-Negative Ratio = {positive_to_negative_ratio_test:.2f}")

Training Set: Positive-to-Negative Ratio = 1.01
Testing Set: Positive-to-Negative Ratio = 0.98


In [2]:
%pip install scikit-learn
import sklearn
%pip install pandas
import pandas as pd


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report


# Create a pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])

# X_train = vectorizer.fit_transform(train_data)
X_train = train_data
y_train = []
for review in train_data:
    if review in positive_data:
        y_train.append("pos")
    else: y_train.append("neg")

# X_test = vectorizer.fit_transform(test_data)
X_test = test_data
y_test = []
for review in test_data:
    if review in positive_data:
        y_test.append("pos")
    else: y_test.append("neg")

# Train the pipeline using the training data
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test) 
accuracy = accuracy_score(y_test, y_pred)

precision_positive = precision_score(y_test, y_pred, pos_label="pos")
precision_negative = precision_score(y_test, y_pred, pos_label="neg")
recall_positive = recall_score(y_test, y_pred, pos_label="pos")
recall_negative = recall_score(y_test, y_pred, pos_label="neg")

print("Accuracy:", accuracy)
print("Precision (Positive):", precision_positive)
print("Precision (Negative):", precision_negative)
print ("Recall (Positive):", recall_positive)
print ("Recall (Negative):", recall_negative)
# Generate classification report
classification_report = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report)

Accuracy: 0.8462666666666666
Precision (Positive): 0.8695527932561794
Precision (Negative): 0.8258967629046369
Recall (Positive): 0.8137451530953336
Recall (Negative): 0.8786065682754953
Classification Report:
               precision    recall  f1-score   support

         neg       0.83      0.88      0.85      7521
         pos       0.87      0.81      0.84      7479

    accuracy                           0.85     15000
   macro avg       0.85      0.85      0.85     15000
weighted avg       0.85      0.85      0.85     15000



In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report


# Create a pipeline with TfidfVectorizer
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

# Train the pipeline using the training data
pipeline.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = pipeline.predict(X_test)

# Calculate accuracy..., the remaining steps remain the same
accuracy = accuracy_score(y_test, y_pred)
precision_positive = precision_score(y_test, y_pred, pos_label="pos")
precision_negative = precision_score(y_test, y_pred, pos_label="neg")
recall_positive = recall_score(y_test, y_pred, pos_label="pos")
recall_negative = recall_score(y_test, y_pred, pos_label="neg")

print("Accuracy:", accuracy)
print("Precision (Positive):", precision_positive)
print("Precision (Negative):", precision_negative)
print ("Recall (Positive):", recall_positive)
print ("Recall (Negative):", recall_negative)
# Generate classification report
classification_report = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report)

Accuracy: 0.8596
Precision (Positive): 0.8784335821946753
Precision (Negative): 0.8426781420073408
Recall (Positive): 0.8338013103356063
Recall (Negative): 0.8852546203962239
Classification Report:
               precision    recall  f1-score   support

         neg       0.84      0.89      0.86      7521
         pos       0.88      0.83      0.86      7479

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000



In [5]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Create a pipeline with CountVectorizer and LogisticRegression
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression())
])

# Train the pipeline using the training data
pipeline.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = pipeline.predict(X_test)

# Calculate accuracy..., the remaining steps remain the same
accuracy = accuracy_score(y_test, y_pred)
precision_positive = precision_score(y_test, y_pred, pos_label="pos")
precision_negative = precision_score(y_test, y_pred, pos_label="neg")
recall_positive = recall_score(y_test, y_pred, pos_label="pos")
recall_negative = recall_score(y_test, y_pred, pos_label="neg")

print("Accuracy:", accuracy)
print("Precision (Positive):", precision_positive)
print("Precision (Negative):", precision_negative)
print ("Recall (Positive):", recall_positive)
print ("Recall (Negative):", recall_negative)
# Generate classification report
classification_report = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8893333333333333
Precision (Positive): 0.8852111743678008
Precision (Negative): 0.8935141667785685
Recall (Positive): 0.8939697820564246
Recall (Negative): 0.8847227762265656
Classification Report:
               precision    recall  f1-score   support

         neg       0.89      0.88      0.89      7521
         pos       0.89      0.89      0.89      7479

    accuracy                           0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000



In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Create a pipeline with CountVectorizer and LogisticRegression
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression())
])

# Train the pipeline using the training data
pipeline.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = pipeline.predict(X_test)

# Calculate accuracy..., the remaining steps remain the same
accuracy = accuracy_score(y_test, y_pred)
precision_positive = precision_score(y_test, y_pred, pos_label="pos")
precision_negative = precision_score(y_test, y_pred, pos_label="neg")
recall_positive = recall_score(y_test, y_pred, pos_label="pos")
recall_negative = recall_score(y_test, y_pred, pos_label="neg")

print("Accuracy:", accuracy)
print("Precision (Positive):", precision_positive)
print("Precision (Negative):", precision_negative)
print ("Recall (Positive):", recall_positive)
print ("Recall (Negative):", recall_negative)
# Generate classification report
classification_report = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report)

Accuracy: 0.8943333333333333
Precision (Positive): 0.883524206142634
Precision (Negative): 0.905686167304538
Recall (Positive): 0.9077416766947453
Recall (Negative): 0.8809998670389576
Classification Report:
               precision    recall  f1-score   support

         neg       0.91      0.88      0.89      7521
         pos       0.88      0.91      0.90      7479

    accuracy                           0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000



In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 

# Create a pipeline with TfidfVectorizer and LogisticRegression
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1,2))),
    ('classifier', LogisticRegression())
])
# Train the pipeline using the training data
pipeline.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = pipeline.predict(X_test)

# Calculate accuracy..., the remaining steps remain the same
accuracy = accuracy_score(y_test, y_pred)
precision_positive = precision_score(y_test, y_pred, pos_label="pos")
precision_negative = precision_score(y_test, y_pred, pos_label="neg")
recall_positive = recall_score(y_test, y_pred, pos_label="pos")
recall_negative = recall_score(y_test, y_pred, pos_label="neg")

print("Accuracy:", accuracy)
print("Precision (Positive):", precision_positive)
print("Precision (Negative):", precision_negative)
print ("Recall (Positive):", recall_positive)
print ("Recall (Negative):", recall_negative)
# Generate classification report
classification_report = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report)

Accuracy: 0.8960666666666667
Precision (Positive): 0.8839169909208819
Precision (Negative): 0.9089163237311385
Recall (Positive): 0.9112180772830593
Recall (Negative): 0.8809998670389576
Classification Report:
               precision    recall  f1-score   support

         neg       0.91      0.88      0.89      7521
         pos       0.88      0.91      0.90      7479

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline 

# Create a pipeline with CountVectorizer and LogisticRegression
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('classifier', LogisticRegression())
])

# Train the pipeline using the training data
pipeline.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = pipeline.predict(X_test)

# Calculate accuracy..., the remaining steps remain the same
accuracy = accuracy_score(y_test, y_pred)
precision_positive = precision_score(y_test, y_pred, pos_label="pos")
precision_negative = precision_score(y_test, y_pred, pos_label="neg")
recall_positive = recall_score(y_test, y_pred, pos_label="pos")
recall_negative = recall_score(y_test, y_pred, pos_label="neg")

print("Accuracy:", accuracy)
print("Precision (Positive):", precision_positive)
print("Precision (Negative):", precision_negative)
print ("Recall (Positive):", recall_positive)
print ("Recall (Negative):", recall_negative)
# Generate classification report
classification_report = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9046
Precision (Positive): 0.8946750195771339
Precision (Negative): 0.9149632052330335
Recall (Positive): 0.9165663858804654
Recall (Negative): 0.89270043877144
Classification Report:
               precision    recall  f1-score   support

         neg       0.91      0.89      0.90      7521
         pos       0.89      0.92      0.91      7479

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report


# Create a pipeline with TfidfVectorizer
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1,2))),
    ('classifier', MultinomialNB())
])

# Train the pipeline using the training data
pipeline.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = pipeline.predict(X_test)

# Calculate accuracy..., the remaining steps remain the same
accuracy = accuracy_score(y_test, y_pred)
precision_positive = precision_score(y_test, y_pred, pos_label="pos")
precision_negative = precision_score(y_test, y_pred, pos_label="neg")
recall_positive = recall_score(y_test, y_pred, pos_label="pos")
recall_negative = recall_score(y_test, y_pred, pos_label="neg")

print("Accuracy:", accuracy)
print("Precision (Positive):", precision_positive)
print("Precision (Negative):", precision_negative)
print ("Recall (Positive):", recall_positive)
print ("Recall (Negative):", recall_negative)
# Generate classification report
classification_report = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report)

Accuracy: 0.8834
Precision (Positive): 0.904775360271263
Precision (Negative): 0.8643019439535471
Recall (Positive): 0.8562642064447119
Recall (Negative): 0.9103842574125781
Classification Report:
               precision    recall  f1-score   support

         neg       0.86      0.91      0.89      7521
         pos       0.90      0.86      0.88      7479

    accuracy                           0.88     15000
   macro avg       0.88      0.88      0.88     15000
weighted avg       0.88      0.88      0.88     15000



In [10]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report


# Create a pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('classifier', MultinomialNB())
])

# Train the pipeline using the training data
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test) 
accuracy = accuracy_score(y_test, y_pred)

precision_positive = precision_score(y_test, y_pred, pos_label="pos")
precision_negative = precision_score(y_test, y_pred, pos_label="neg")
recall_positive = recall_score(y_test, y_pred, pos_label="pos")
recall_negative = recall_score(y_test, y_pred, pos_label="neg")

print("Accuracy:", accuracy)
print("Precision (Positive):", precision_positive)
print("Precision (Negative):", precision_negative)
print ("Recall (Positive):", recall_positive)
print ("Recall (Negative):", recall_negative)
# Generate classification report
classification_report = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report)

Accuracy: 0.8784
Precision (Positive): 0.8921092774927195
Precision (Negative): 0.8657080498138401
Recall (Positive): 0.8601417301778312
Recall (Negative): 0.8965563090014625
Classification Report:
               precision    recall  f1-score   support

         neg       0.87      0.90      0.88      7521
         pos       0.89      0.86      0.88      7479

    accuracy                           0.88     15000
   macro avg       0.88      0.88      0.88     15000
weighted avg       0.88      0.88      0.88     15000



In [12]:
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 

# Create a pipeline with TfidfVectorizer and LogisticRegression
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1,2))),
    ('classifier', LogisticRegression())
])
# Train the pipeline using the training data
pipeline.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = pipeline.predict(X_test)

# best_medel = y_pred

with open('model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [ ]:
%pip install fasttext
import fasttext

# print(train_data)

model = fasttext.train_supervised(input='train.txt')
y_pred = model.predict(X_test)[0]

y_pred = [label[0].replace('__label__', '') for label in y_pred]

accuracy = accuracy_score(y_test, y_pred)
precision_positive = precision_score(y_test, y_pred, pos_label='positive')
precision_negative = precision_score(y_test, y_pred, pos_label='negative')
recall_positive = recall_score(y_test, y_pred, pos_label='positive')
recall_negative = recall_score(y_test, y_pred, pos_label='negative')
classification_report = classification_report(y_test, y_pred)

In [11]:
# import numpy as np

# df = pd.DataFrame(train_data)
# for review in train_data:
#     if review in positive_data:
#         df.insert(2,"label","pos")
#     else: df.insert(2,"label","neg")
# print(df)

# path = r'/Users/shun/Desktop/IERG4080/train.txt'
# with open(path, 'a') as f:
#     df = df.to_string(header=False, index=False)
#     f.write(df)

# for filename in os.listdir(train_data):
#     with open(os.path.join(train_data, filename), "r", encoding="utf-8") as file:
#         review = file.read()
#         reviews.append(review)
#         if review in positive_data:
#             labels.append("pos")
#         else: labels.append("neg")

# # Create a DataFrame, whatever you like to store the combined data
# data = pd.DataFrame({"review": reviews, "sentiment": labels})
# # Optionally, you can save the combined dataset to a CSV file
# data.to_csv("traindata.csv", index=False) # Replace with the desired output file path


IndexError: index 2 is out of bounds for axis 0 with size 1